In [2]:
pip install google-api-core


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -r requirements.txt

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached cachetools-4.2.0-py3-none-any.whl (12 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached cffi-1.14.4-cp39-cp39-win_amd64.whl (179 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached dialogflow-1.1.0-py2.py3-none-any.whl (354 kB)
  Using cached gcloud-0.17.0.tar.gz (458 kB)
  Using cached google_api_core-1.24.1-py2.py3-none-any.whl (92 kB)
  Using cached google_auth-1.24.0-py2.py3-none-any.whl (114 kB)
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
  Using cached grpcio-1.34.0-cp39-cp39-win_amd64.whl (2.9 MB)
  Using cached httplib2-0.18.1-py3-none-any.whl (95 kB)
  Using cached joblib-1.0.0-py3-none-any.whl (302 kB)
  Using cached Js2Py-0.70-py3-none-any.whl (605 kB)
  Using cached jws-0.1.3.tar.gz (8.1 kB)
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Using cached llvmlite-0.35.0.tar.gz (121 kB)
  Using cached numpy-1.19

ERROR: Cannot install -r requirements.txt (line 12), -r requirements.txt (line 27), -r requirements.txt (line 28) and -r requirements.txt (line 39) because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [5]:
pip install requests==2.25.1


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [7]:
import argparse
import os
import numpy as np
import logging
import re
import timeit
from pathlib import Path
import audiofile

import methods
from methods import Reader
from methods.feature_engineer import FeatureEngineer


In [16]:
import wave
wave.open('data/bp/bp-11.wav')



Error: file does not start with RIFF id

In [20]:
import subprocess
subprocess.call(['ffmpeg', '-i', 'data/bp/bp-2.mp3', 'data/bp/bp-2.wav'])


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [10]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--load_path',
                        default='data/')
    parser.add_argument('--save_path',
                        default='output/model/')
    parser.add_argument('--log_path',
                        default='logs/')

    # Arguments
    args = argparse.ArgumentParser()
    #load_path = args.load_path
    load_path = Path("data/")
    #save_path = args.save_path
    save_path = Path("output/model/")
    #log_path = args.log_path
    log_path = Path('methods/logs/')

    ####################################################################################################################
    # Set up logging
    ####################################################################################################################

    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s',
                        datefmt='%Y-%m-%d %I:%M:%S %p',
                        filename=os.path.join(log_path, 'logs_pc_methods_feat_eng.log'),
                        filemode='w',
                        level=logging.INFO)

    ####################################################################################################################
    # READ FILES IN SUB-FOLDERS of load_path and FEATURE ENGINEERING
    ####################################################################################################################

    # list load_path sub-folders
    #regex = re.compile(r'^[0-9]')
    #directory_list = [i for i in os.listdir(load_path) if regex.search(i)]
    directory_list = [i for i in os.listdir(load_path)]
    
    # initialize empty array for features
    X = np.empty([1, 18])

    # initialise empty array for labels
    y = []

    logging.info('Creating training set...')
    start = timeit.default_timer()

    # iteration on sub-folders
    for directory in directory_list:

        # Instantiate FeatureEngineer
        feature_engineer = FeatureEngineer(label=directory)

        file_list = os.listdir(os.path.join(load_path, directory))

        # iteration on audio files in each sub-folder
        for audio_file in file_list:
            file_reader = Reader(os.path.join(load_path, directory, audio_file))
            data, sample_rate = file_reader.read_audio_file()
            print("yes")
            avg_features, label = feature_engineer.feature_engineer(audio_data=data)

            X = np.concatenate((X, avg_features), axis=0)
            y.append(label)

    # X.shape is (401, 18) as I'm not using indexing. First line is made of zeros and is to be removed
    X = X[1:, :]

    stop = timeit.default_timer()
    logging.info('Time taken for reading files and feature engineering: {0}'.format(stop - start))

    ####################################################################################################################
    # SAVE
    ####################################################################################################################

    logging.info('Saving training set...')

    # Save to numpy binary format
    np.save(os.path.join(save_path, 'dataset.npy'), X)

    np.save(os.path.join(save_path, 'labels.npy'), y)

    logging.info('Saved! {0}'.format(os.path.join(save_path, 'dataset.npy')))
    logging.info('Saved! {0}'.format(os.path.join(save_path, 'labels.npy')))

In [11]:
if __name__ == '__main__':
    main()

NoBackendError: 